In [7]:
import requests
import pandas as pd
from PIL import Image
from io import BytesIO
import os
from concurrent.futures import ThreadPoolExecutor

# Google Drive 연결
from google.colab import drive
drive.mount('/content/drive')

# 1. movie_title.csv 파일 불러오기
file_path = '/content/movie_title.csv'  # 업로드한 movie_title.csv 파일 경로
movies_data = pd.read_csv(file_path)

# 데이터 확인
print("불러온 데이터:")
print(movies_data.head())

# TMDb API 키
API_KEY = '8adcb8011cfeca4b9429d3ad38b5360b'

# 2. TMDb API를 호출해 영화 포스터 URL 가져오는 함수
def get_movie_poster(title):
    try:
        url = f"https://api.themoviedb.org/3/search/movie?api_key={API_KEY}&query={title}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if data['results']:
                poster_path = data['results'][0].get('poster_path')  # 첫 번째 결과의 포스터 경로
                if poster_path:
                    return f"https://image.tmdb.org/t/p/w500{poster_path}"
    except Exception as e:
        print(f"Error fetching poster for {title}: {e}")
    return None

# 3. 병렬 처리로 포스터 URL 가져오기
def get_posters_parallel(titles, max_workers=10):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        return list(executor.map(get_movie_poster, titles))

# 병렬 처리 실행 (max_workers=10으로 설정)
print("병렬 처리로 포스터 URL 가져오는 중...")
movies_data['poster_url'] = get_posters_parallel(movies_data['movie_title'].tolist(), max_workers=10)

# 데이터프레임 확인
print("포스터 URL이 추가된 데이터:")
print(movies_data[['movie_title', 'poster_url']].head())

# 4. 포스터 이미지를 다운로드하고 크기 조정하는 함수
def resize_image(image_url, size=(200, 300)):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            img_resized = img.resize(size)
            return img_resized
    except Exception as e:
        print(f"이미지 다운로드 실패: {image_url}, 에러: {e}")
    return None

# 5. 병렬 처리로 포스터 이미지 다운로드 및 저장
def download_and_save_posters(data, save_folder, max_workers=10):
    os.makedirs(save_folder, exist_ok=True)  # 폴더 생성

    def process_row(row):
        if row['poster_url']:
            img_resized = resize_image(row['poster_url'])
            if img_resized:
                file_name = f"{save_folder}{row['movie_title'].replace('/', '_')}.jpg"
                img_resized.save(file_name)
                return file_name
        return None

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        file_paths = list(executor.map(process_row, data.to_dict('records')))
    return file_paths

# 포스터 이미지 저장 폴더 설정
poster_save_folder = "/content/drive/MyDrive/posters/"
print("병렬 처리로 포스터 이미지를 저장하는 중...")
movies_data['poster_path'] = download_and_save_posters(movies_data, poster_save_folder, max_workers=10)


# 6. 최종 데이터 저장
final_data_path = '/content/drive/MyDrive/movies_with_posters.csv'
movies_data.to_csv(final_data_path, index=False)

# 결과 출력
print("포스터 경로가 추가된 최종 데이터:")
print(movies_data[['movie_title', 'poster_url', 'poster_path']].head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
불러온 데이터:
                                         movie_title
0  Percy Jackson & the Olympians: The Lightning T...
1                                        Please Give
2                                                 10
3                    12 Angry Men (Twelve Angry Men)
4                       20,000 Leagues Under The Sea
병렬 처리로 포스터 URL 가져오는 중...
포스터 URL이 추가된 데이터:
                                         movie_title  \
0  Percy Jackson & the Olympians: The Lightning T...   
1                                        Please Give   
2                                                 10   
3                    12 Angry Men (Twelve Angry Men)   
4                       20,000 Leagues Under The Sea   

                                          poster_url  
0  https://image.tmdb.org/t/p/w500/8wmqmvJknPv6Wp...  
1  https://image.tmdb.org/t/p/w500/kft7oy2SoImjDU...  